# **RAG-Based-Tableau-Course-Q-A-Chatbot** 

In [1]:
import warnings 
warnings.filterwarnings('ignore')

In [2]:
try:
    with open('google_api_key.txt') as f:
        key = f.read()
except FileNotFoundError:
    print("API-KEY not found!!")

In [3]:
from langchain_google_genai import (ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings)
from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_core.messages import SystemMessage
from langchain_core.prompts import (PromptTemplate, HumanMessagePromptTemplate, 
                                    SystemMessagePromptTemplate, ChatPromptTemplate)
from langchain_core.runnables import (RunnableLambda, RunnableParallel, RunnablePassthrough, chain)
from langchain_core.output_parsers import StrOutputParser
from langchain_chroma import Chroma
from langchain_text_splitters import (MarkdownHeaderTextSplitter, TokenTextSplitter)

## Load the Course Transcript

In [4]:
loader_pdf = PyPDFLoader('Introduction_to_Tableau.pdf')

In [5]:
docs_list = loader_pdf.load()

In [6]:
len(docs_list)

49

In [7]:
docs_list

[Document(metadata={'source': 'Introduction_to_Tableau.pdf', 'page': 0}, page_content="# Introduction to Tableau \n## Welcome to Tableau \nHi, everyone. \nI'm Ned and I'll be your instructor for this \ncourse. \nTableau is an invaluable tool. \nOne needs to learn on their journey to become a \nsuccessful business intelligence analyst or \ndata scientist. \nThe art of these professions is storytelling \nusing data to tell stories and convince top \nmanagement of the right course of action. \nBy completing this part of the program, you \nwill know how to create charts and dashboards \nin tableaux. \nThis is an essential step on your way to a data \nscientist role. \n \n## Why use Tableau: Make your data make an impact \nTableau has grown to become one of the most \npopular business intelligence tools in the \nentire world. \nIt is A B I software that allows non technical \nusers to visualize their data and work with it \nalmost immediately lowering, \nknow how barriers dramatically in th

In [8]:
string_list_concat = "\n".join([doc.page_content for doc in docs_list])

In [9]:
string_list_concat

"# Introduction to Tableau \n## Welcome to Tableau \nHi, everyone. \nI'm Ned and I'll be your instructor for this \ncourse. \nTableau is an invaluable tool. \nOne needs to learn on their journey to become a \nsuccessful business intelligence analyst or \ndata scientist. \nThe art of these professions is storytelling \nusing data to tell stories and convince top \nmanagement of the right course of action. \nBy completing this part of the program, you \nwill know how to create charts and dashboards \nin tableaux. \nThis is an essential step on your way to a data \nscientist role. \n \n## Why use Tableau: Make your data make an impact \nTableau has grown to become one of the most \npopular business intelligence tools in the \nentire world. \nIt is A B I software that allows non technical \nusers to visualize their data and work with it \nalmost immediately lowering, \nknow how barriers dramatically in the past. \nBusiness analysts needed the help of it \npersonnel who could assist them in

In [10]:
print(string_list_concat)

# Introduction to Tableau 
## Welcome to Tableau 
Hi, everyone. 
I'm Ned and I'll be your instructor for this 
course. 
Tableau is an invaluable tool. 
One needs to learn on their journey to become a 
successful business intelligence analyst or 
data scientist. 
The art of these professions is storytelling 
using data to tell stories and convince top 
management of the right course of action. 
By completing this part of the program, you 
will know how to create charts and dashboards 
in tableaux. 
This is an essential step on your way to a data 
scientist role. 
 
## Why use Tableau: Make your data make an impact 
Tableau has grown to become one of the most 
popular business intelligence tools in the 
entire world. 
It is A B I software that allows non technical 
users to visualize their data and work with it 
almost immediately lowering, 
know how barriers dramatically in the past. 
Business analysts needed the help of it 
personnel who could assist them in gathering 
raw data and pre

## Split the Course Transcript with MarkdownHeaderTextSplitter

In [11]:
md_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=[("#", "Section Title"),
                                                             ("##", "Course Title")])

In [12]:
docs_list_md_split = md_splitter.split_text(string_list_concat)

In [13]:
docs_list_md_split

[Document(metadata={'Section Title': 'Introduction to Tableau', 'Course Title': 'Welcome to Tableau'}, page_content="Hi, everyone.\nI'm Ned and I'll be your instructor for this\ncourse.\nTableau is an invaluable tool.\nOne needs to learn on their journey to become a\nsuccessful business intelligence analyst or\ndata scientist.\nThe art of these professions is storytelling\nusing data to tell stories and convince top\nmanagement of the right course of action.\nBy completing this part of the program, you\nwill know how to create charts and dashboards\nin tableaux.\nThis is an essential step on your way to a data\nscientist role."),
 Document(metadata={'Section Title': 'Introduction to Tableau', 'Course Title': 'Why use Tableau: Make your data make an impact'}, page_content="Tableau has grown to become one of the most\npopular business intelligence tools in the\nentire world.\nIt is A B I software that allows non technical\nusers to visualize their data and work with it\nalmost immediatel

### Create a Chain to Correct the Course Transcript

In [14]:
string_list_split = [doc.page_content for doc in docs_list_md_split]

In [ ]:
PROMPT_FORMATTING_S = '''Improve the following Tableau lecture transcript by:
- Splitting the text into meaningful paragraphs
- Correcting any misplaced punctuation
- Fixing mistranscribed words (e.g., changing 'tableaux' to 'Tableau')"
'''

PROMPT_TEMPLATE_FORMATTING_H = '''This is the transcript:
{lecture_transcript}
'''

In [ ]:
system_prompt = SystemMessage(content=PROMPT_FORMATTING_S)
human_prompt = HumanMessagePromptTemplate.from_template(PROMPT_TEMPLATE_FORMATTING_H)
chat_prompt_template_formatting = ChatPromptTemplate.from_messages([system_prompt,
                                                                    human_prompt])

In [17]:
chat = ChatGoogleGenerativeAI(api_key=key, 
                             model='models/gemini-1.5-flash',
                             temperature=0,
                             model_kwargs = {'seed':365})

In [18]:
str_output_parser = StrOutputParser()

In [19]:
chain_formatting = chat_prompt_template_formatting | chat | str_output_parser

In [20]:
# import time
# def process_lectures_synchronously(Lectures):
#     formatted_lectures = []
#     for lecture in Lectures:
#         result = chain_formatting.invoke({'lecture_transcript':lecture})
#         formatted_lectures.append(result)
#         time.sleep(1) # to avoid ResourceExhausted: 429 Resource has been exhausted 
#     return formatted_lectures

In [ ]:
from tqdm import tqdm
import time

def process_lectures_synchronously(Lectures):
    formatted_lectures = []
    # Wrap the loop with tqdm for progress tracking
    for lecture in tqdm(Lectures, desc="Processing lectures", unit="lecture"):
        # Invoke the processing logic
        result = chain_formatting.invoke({'lecture_transcript': lecture})
        formatted_lectures.append(result)
        time.sleep(1)  # Avoid resource exhaustion (Rate limit)
    return formatted_lectures

In [ ]:
# import time

# def process_with_retries(lectures, retries=3, delay=5):
#     formatted_lectures = []
#     for lecture in lectures:
#         attempt = 0
#         while attempt < retries:
#             try:
#                 result = chain_formatting.invoke({"lecture_transcript": lecture})
#                 formatted_lectures.append(result)
#                 break
#             except Exception as e:
#                 print(f"Attempt {attempt + 1} failed. Retrying in {delay} seconds...")
#                 time.sleep(delay)
#                 attempt += 1
#         else:
#             print("Max retries reached.")
#     return formatted_lectures


In [ ]:
formatted_lectures = process_lectures_synchronously(string_list_split)

In [ ]:
formatted_lectures

In [ ]:
# Save data to a pickle file
import pickle
def save_to_pickle(data, filename="formatted_lectures.pkl"):
    with open(filename, "wb") as f:
        pickle.dump(data, f)

In [ ]:
filename = "formatted_lectures.pkl"
save_to_pickle(formatted_lectures, filename)

In [22]:
import pickle
def load_from_pickle(filename="formatted_lectures.pkl"):
    with open(filename, "rb") as f:
        return pickle.load(f)

In [23]:
filename = "formatted_lectures.pkl"
formatted_lectures = load_from_pickle(filename)

In [24]:
formatted_lectures

["Hi, everyone. I'm Ned, and I'll be your instructor for this course.  Tableau is an invaluable tool that's essential to learn on your journey to becoming a successful business intelligence analyst or data scientist.\n\nThe art of these professions lies in storytelling—using data to craft compelling narratives and convince top management of the best course of action.\n\nBy completing this part of the program, you will learn how to create charts and dashboards in Tableau. This is a crucial step on your path to a data scientist role.\n",
 "Tableau has grown to become one of the most popular business intelligence tools worldwide. It is a BI software that allows non-technical users to visualize and work with their data almost immediately, dramatically lowering the knowledge barriers present in the past.  Previously, business analysts needed the help of IT personnel to gather and preprocess raw data before visualization could begin.\n\nThe advent of Tableau democratized this process, allowi

##### on comparing formatted_lectures with string_list_split course transcript has been corrected.
##### substituting the lecture content of the Document objects in the docs_list_md_split list with these new texts.

In [25]:
for doc, formatted_text in zip(docs_list_md_split, formatted_lectures):
    doc.page_content = formatted_text

In [26]:
docs_list_md_split

[Document(metadata={'Section Title': 'Introduction to Tableau', 'Course Title': 'Welcome to Tableau'}, page_content="Hi, everyone. I'm Ned, and I'll be your instructor for this course.  Tableau is an invaluable tool that's essential to learn on your journey to becoming a successful business intelligence analyst or data scientist.\n\nThe art of these professions lies in storytelling—using data to craft compelling narratives and convince top management of the best course of action.\n\nBy completing this part of the program, you will learn how to create charts and dashboards in Tableau. This is a crucial step on your path to a data scientist role.\n"),
 Document(metadata={'Section Title': 'Introduction to Tableau', 'Course Title': 'Why use Tableau: Make your data make an impact'}, page_content="Tableau has grown to become one of the most popular business intelligence tools worldwide. It is a BI software that allows non-technical users to visualize and work with their data almost immediate

## **Split the Lectures with TokenTextSplitter**

In [27]:
tokensplit = TokenTextSplitter(
    encoding_name="cl100k_base",
    chunk_size = 500,
    chunk_overlap = 50
)

In [28]:
docs_list_token_split = tokensplit.split_documents(docs_list_md_split)

## **Create Embeddings, Vector Store and Retriever**

In [29]:
embedding = GoogleGenerativeAIEmbeddings(google_api_key=key,
                                         model="models/text-embedding-004")

In [30]:
vectorestore = Chroma.from_documents(persist_directory="./intro-to-tableau",
                                    documents=docs_list_token_split,
                                    embedding=embedding)

In [31]:
retriever = vectorestore.as_retriever(search_type="mmr", 
                                     search_kwargs = {'k':2,
                                                     'lambda_mult':0.7})

## **Create Prompts and Prompt Templates for the Q&A Chatbot Chain**

In [32]:
PROMPT_CREATING_QUESTION = '''Lecture: {question_lecture}
Title: {question_title}
Body: {question_body}'''

PROMPT_RETRIEVING_S = '''You will receive a question from a student taking a Tableau course, which includes a title and a body. 
The corresponding lecture will also be provided.

Answer the question using only the provided context.

At the end of your response, include the section and lecture names where the context was drawn from, formatted as follows: 
Resources: 
Section: *Section Title*, Lecture: *Lecture Title* 
...
Replace *Section Title* and *Lecture Title* with the appropriate titles.'''

PROMPT_TEMPLATE_RETRIEVING_H = '''This is the question:
{question}

This is the context:
{context}'''

prompt_creating_question = PromptTemplate.from_template(template=PROMPT_CREATING_QUESTION)
prompt_retrieving_s = SystemMessage(content=PROMPT_RETRIEVING_S)
prompt_template_retrieving_h = HumanMessagePromptTemplate.from_template(template=PROMPT_TEMPLATE_RETRIEVING_H)
chat_prompt_template_retrieving = ChatPromptTemplate([prompt_retrieving_s,
                                                     prompt_template_retrieving_h])

## **First Version of Q&A Chainbot chain**

In [33]:
chain_retrieving = ( prompt_creating_question
                     |RunnableLambda(lambda x: x.text)
                     |{'context':retriever,
                       'question' : RunnablePassthrough()}
                     |chat_prompt_template_retrieving 
                     |chat
                     |str_output_parser)

In [34]:
inputs = {"question_lecture": "Adding a custom calculation",
          "question_title": "Why are we using SUM here? It's unclear to me.",
          "question_body": "This question refers to calculating the GM%."}

In [35]:
result = chain_retrieving.invoke(inputs)

In [36]:
print(result)

The `SUM` function is used because without it, the calculation `Gross Profit / Revenue` is performed on individual data points for each month, rather than the total `Gross Profit` and total `Revenue` across all months.  This leads to incorrect results.  Using `SUM(Gross Profit) / SUM(Revenue)` ensures that the total gross profit is divided by the total revenue, giving the correct gross margin percentage.

Resources: 
Section: Tableau Functionalities, Lecture: Adding a custom calculation




## **Creating a Runnable function to format the context**

In [54]:
@chain
def format_context(dictionary):
    formatted_string = ""
    retrieved_list = dictionary["context"]
    for i in range(len(retrieved_list)):
        formatted_string += f"""
        Document {i+1}
        Section Title : {retrieved_list[i].metadata["Section Title"]}
        Lecture Title : {retrieved_list[i].metadata["Course Title"]}
        Content : {retrieved_list[i].page_content}
        """
    new_dictionary = {"context" : formatted_string, 
                      "question" : dictionary["question"]}
    return new_dictionary

In [55]:
chain_retrieving_formatted = ( prompt_creating_question
                     |RunnableLambda(lambda x: x.text)
                     |{'context':retriever,
                       'question' : RunnablePassthrough()}
                     |format_context
                     |chat_prompt_template_retrieving 
                     |chat
                     |str_output_parser)

In [56]:
querry = {"question_lecture": "Adding a custom calculation",
         "question_title": "Why are we using SUM here? It's unclear to me.",
         "question_body": "This question refers to calculating the GM%."}

In [57]:
result_formatted = chain_retrieving_formatted.invoke(querry)

In [58]:
print(result_formatted)

The `SUM` function is used because without it, the calculation `Gross Profit / Revenue` is performed on individual data points for each month, rather than the total `Gross Profit` and total `Revenue` across all months.  This leads to incorrect results.  Using `SUM(Gross Profit) / SUM(Revenue)` ensures that the total gross profit is divided by the total revenue, giving the correct gross margin percentage.

Resources: 
Section: Tableau Functionalities, Lecture: Adding a custom calculation




## **Stream the Response**

In [59]:
result_streamed = chain_retrieving_formatted.stream(querry)

In [60]:
result_streamed

<generator object RunnableSequence.stream at 0x000001A8F21A7B50>

In [61]:
for chunk in result_streamed:
    print(chunk, end="")

The `SUM` function is used because without it, the calculation `Gross Profit / Revenue` is performed on individual data points for each month, rather than the total `Gross Profit` and total `Revenue` across all months.  This leads to incorrect results.  Using `SUM(Gross Profit) / SUM(Revenue)` ensures that the total gross profit is divided by the total revenue, giving the correct gross margin percentage.

Resources: 
Section: Tableau Functionalities, Lecture: Adding a custom calculation

